In [ ]:
!fusermount -u drive

## 드라이브 마운팅 & 워킹 디렉토리 변경

In [ ]:
from google.colab import drive
drive.mount('/content/drive/MyDrive')

Mounted at /content/drive/MyDrive


In [ ]:
import os

path = '/content/drive/MyDrive/MyDrive/NLP-StockMarket'
os.chdir(path)

## 필요 모듈 import

In [ ]:
!pip install konlpy   # 코랩에서만 실행

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 5.4 MB/s 
     |████████████████████████████████| 453 kB 54.4 MB/s 


In [ ]:
import pandas as pd
from tqdm import tqdm
from konlpy.tag import *
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import re
from datetime import datetime, timedelta
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# 한글 깨짐 방지
import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

# - 기호 깨짐 방지
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
okt = Okt()

## 종목 선택, 뉴스 & 토론방 & 유튜브 데이터 통합

In [ ]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [ ]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col=0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col=0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col=0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col=0)
youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col=0)

In [ ]:
naver_talks['Date'] = pd.to_datetime(naver_talks['Date'])
daum_talks['Date'] = pd.to_datetime(daum_talks['Date'])

In [ ]:
# 데이터 통합
news_df = pd.concat([naver_news, daum_news, naver_talks, daum_talks, youtube])

# 'Date'열의 데이터 타입이 int여서 datetime으로 변환
news_df['Date'] = pd.to_datetime(news_df['Date'].astype(str))     # str(변수) 는 자체 내장함수, 변수 하나씩밖에 적용 안됨.

# 합쳐진 데이터의 인덱스 재설정
news_df.sort_values('Date', ignore_index=True, inplace=True)

news_df

,Date,Title
0,2021-01-01,서학개미 사로잡은 美 ESG ETF 올해도 高高할까
1,2021-01-01,신기록 쏟아낸 개미 순매수 63兆 예탁금 65兆 빚투 19兆
2,2021-01-01,2020년 재테크 성적표 1위는 주식 달러는 마이너스
3,2021-01-01,반도체 장비기업 ASML홀딩 급등 지금 매수해도 괜찮을까
4,2021-01-01,그래픽 GDP대비 코스피 시가총액 비율
...,...,...
679641,2022-07-05,노터스 주가전망 노터스 주가 노터스 주식 노터스 주식전망 노터스 주가전망 노터스...
679642,2022-07-05,한국비엔씨주가 한국비엔씨주가분석 한국비엔씨주식 한국비엔씨신주인수권 한국비엔...
679643,2022-07-05,씨아이에스 호재 떴다기사나오면 늦어요 삼성sdi 씨아이에스 전망 씨아이에스 주가 전...
679644,2022-07-05,버킷스튜디오 슈팅 나올 신호 그게 뭔데


In [ ]:
# 주가 데이터 시작이 1월 4일 부터여서 뉴스도 1월 4일 이전은 슬라이싱으로 없애기
news_df = news_df[news_df[news_df['Date'] == '2021-01-04'].index[0] : ]
news_df.head(2)

,Date,Title
639,2021-01-04,속보 새해 첫 거래일 코스피 0 04 오른 2874 50 출발
640,2021-01-04,대웅제약 코로나 치료제 임상3상 시험 시작 주가 상승


## KRX에서 받은 csv로 가져오기

In [ ]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', usecols = ['일자', '등락률'])
stock_df['일자'] = pd.to_datetime(stock_df['일자'])
stock_df.head(3)

,일자,등락률
0,2021-01-04,6.85
1,2021-01-05,2.24
2,2021-01-06,-0.87


## 주가 데이터

In [ ]:
start = str(stock_df.iloc[0, 0])
end = str(stock_df.iloc[-1, 0])
print('start : ', start)
print('end : ', end)

start :  2021-01-04 00:00:00
end :  2022-06-30 00:00:00


In [ ]:
# multi 분류
stock_df['updown'] = 0

stock_df.loc[stock_df.query('등락률 > 1').index, 'updown'] = 1
stock_df.loc[stock_df.query('등락률 < -1').index, 'updown'] = -1

stock_df['updown'].value_counts()

 0    149
 1    111
-1    109
Name: updown, dtype: int64

In [ ]:
# 뉴스 일자 열 추가 : 예측대상(= 익일의 주가)의 일자와 맞추기 위함

news_df['일자'] = news_df['Date'] + timedelta(days = 1)
news_df.head(3)

,Date,Title,일자
639,2021-01-04,속보 새해 첫 거래일 코스피 0 04 오른 2874 50 출발,2021-01-05
640,2021-01-04,대웅제약 코로나 치료제 임상3상 시험 시작 주가 상승,2021-01-05
641,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05


# 뉴스 데이터 & 주가 데이터 합치기

In [ ]:
df = news_df.merge(stock_df)

# df 칼럼명 : Date, Title, 주가의 날짜(구 : 일자), 등락률(y), updown
df.columns = [df.columns[0], df.columns[1], '주가의 날짜', '등락률(y)', 'updown']

# 기사 제목 중복 제거
df.drop_duplicates('Title', inplace = True, ignore_index = True)

print(len(df))
df['updown'].value_counts()

431484


 0    171003
 1    142256
-1    118225
Name: updown, dtype: int64

In [ ]:
df.to_csv('./data/[Model6]merged_df.csv', encoding = 'utf-8-sig')

df = pd.read_csv('./data/[Model6]merged_df.csv', index_col = 0)
df['updown'].value_counts()

 0    171003
 1    142256
-1    118225
Name: updown, dtype: int64

# 감성사전 불러오기

In [ ]:
sentiment_csv = pd.read_csv('./data/sentiment dictionary.csv', index_col = 0)
sentiment_csv.head()

,pos,mid,neg
0,방긋,아직,회의적
1,상회,보통,바닥
2,신선,vs,이탈
3,신박,중립,떨어지
4,투혼,관망,안좋게


In [ ]:
pos_li = sentiment_csv['pos'].dropna().values
mid_li = sentiment_csv['mid'].dropna().values
neg_li = sentiment_csv['neg'].dropna().values

print(f'pos_li : {len(pos_li)} 개')
print(f'mid_li : {len(mid_li)} 개')
print(f'neg_li : {len(neg_li)} 개')

pos_li : 641 개
mid_li : 53 개
neg_li : 959 개


# 감성지수 계산하는 함수 : sentimental_score()

In [ ]:
def sentimental_score(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['Mid'] = 0
    
    # 감성 지수는 긍정 : 1, 중립 : 0, 부정 : -1, 해당 데이터 제외 : 999
    df_result['감성지수'] = 999    
    
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        df_result.loc[df_result[df_result['Title'].str.contains(pos) == True].index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        df_result.loc[df_result[df_result['Title'].str.contains(neg) == True].index, 'Neg'] = 1
    
    print('중립 단어 검색중')
    for mid in tqdm(mid_li) :
        df_result.loc[df_result[df_result['Title'].str.contains(mid) == True].index, 'Mid'] = 1
    
    df_result.to_csv('./data/[Model6]pos_neg_mid_added_df_result.csv', encoding = 'utf-8-sig')



    # 모든 종류의 단어가 검출 되면 제외
    df_result.loc[df_result[(df_result['Pos'] == 1) & (df_result['Neg'] == 1) & (df_result['Mid'] == 1)].index, '감성지수'] = 999
    
    # 중립 단어가 검출되면 중립
    df_result.loc[df_result[(df_result['Pos'] == 1) & (df_result['Neg'] == 0) & (df_result['Mid'] == 1)].index, '감성지수'] = 0
    df_result.loc[df_result[(df_result['Pos'] == 0) & (df_result['Neg'] == 1) & (df_result['Mid'] == 1)].index, '감성지수'] = 0
    df_result.loc[df_result[(df_result['Pos'] == 0) & (df_result['Neg'] == 0) & (df_result['Mid'] == 1)].index, '감성지수'] = 0

    # 긍 부 중립 모두 검출 안될 경우는 일단 0이라고 했음
    df_result.loc[df_result[(df_result['Pos'] == 0) & (df_result['Neg'] == 0) & (df_result['Mid'] == 0)].index, '감성지수'] = 0

    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result[(df_result['Pos'] == 1) & (df_result['Neg'] == 0) & (df_result['Mid'] == 0)].index, '감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result[(df_result['Pos'] == 0) & (df_result['Neg'] == 1) & (df_result['Mid'] == 0)].index, '감성지수'] = -1
    
    # 긍정, 부정 단어가 둘 다 있으면 전 날 또는 당일 주가의 등락률을 보고 결정
    print('긍정 부정 둘 다 있는 경우 처리중')
    for i in tqdm(df_result.loc[df_result[(df_result['Pos'] == 1) & (df_result['Neg'] == 1) & (df_result['Mid'] == 0)].index].index) : 
        updown = 999 # 등락률을 뜻하는 updown
        
        # 해당 Title의 어제 주가가 있으면 선택
        # if sum((df_result.loc[i,'Date'] - timedelta(days = 1)) == stock_df['일자']) == 1 :  
        if sum(pd.to_datetime(df_result.loc[i, 'Date']) - timedelta(days = 1) == stock_df['일자']) == 1 :
            updown = stock_df[stock_df['일자'] == (pd.to_datetime(df_result.loc[i,'Date']) - timedelta(days = 1))]['등락률'].values[0]
        
        # 어제 주가는 없지만 당일이 있으면 당일을 선택
        # elif sum(df_result.loc[i,'Date'] == stock_df['일자']) == 1 : 
        elif sum(pd.to_datetime(df_result.loc[i,'Date']) == stock_df['일자']) == 1 :
            updown = stock_df[stock_df['일자'] == pd.to_datetime(df_result.loc[i,'Date'])]['등락률'].values[0]

        # 어제와 오늘의 주가도 없다면 이전의 주가를 찾아 탐색
        else :
            j = 2 
            while True :
                # if sum((df_result.loc[i,'Date'] - timedelta(days = j)) == stock_df['일자']) == 1 :
                if sum((pd.to_datetime(df_result.loc[i, 'Date']) - timedelta(days = j)) == stock_df['일자']) == 1 :
                    updown = stock_df[stock_df['일자'] == (pd.to_datetime(df_result.loc[i,'Date']) - timedelta(days = j))]['등락률'].values[0]
                    break
                j += 1
        
        # 절댓값이 0보다 낮은 등락률은 변화가 없다고 판단
        if updown > 1 :
            # df_result['감성지수'][i] = 1
            df_result.loc[i, '감성지수'] = 1
        elif updown < 1 :
            # df_result['감성지수'][i] = -1
            df_result.loc[i, '감성지수'] = -1
        else :
            # df_result['감성지수'][i] = 0
            df_result.loc[i, '감성지수'] = 0
    print('df_result["감성지수"] : \n',  df_result['감성지수'].value_counts())

    return df_result

In [ ]:
df_result = sentimental_score(df)

긍정 단어 검색중


100%|██████████| 641/641 [02:41<00:00,  3.98it/s]


부정 단어 검색중


100%|██████████| 959/959 [04:03<00:00,  3.94it/s]


중립 단어 검색중


100%|██████████| 53/53 [00:13<00:00,  4.00it/s]


긍정 부정 둘 다 있는 경우 처리중


100%|██████████| 93069/93069 [07:50<00:00, 197.84it/s]

df_result["감성지수"] : 
  1      215562
-1      130745
 0       79152
 999      6025
Name: 감성지수, dtype: int64


In [ ]:
df_result['감성지수'].value_counts()

 1      215562
-1      130745
 0       79152
 999      6025
Name: 감성지수, dtype: int64

In [ ]:
df_result = pd.read_csv('./data/[Model6]감성지수_added_df_result.csv', index_col = 0)
df_result

In [ ]:
len(df_result)

431484

In [ ]:
df_result = df_result.drop(df_result[df_result['감성지수'] == 999].index)
len(df_result)

425459

In [ ]:
df_result.to_csv('./data/[Model6]감성지수_added_df_result.csv', encoding = 'utf-8-sig')

# 예측 모델 적용

In [ ]:
df_result['tokenized'] = 0
df_result.dropna(how='any',inplace= True)
df_result['tokenized'] = df_result['Title'].apply(okt.morphs)   # 또는 자유롭게 라이브러리 사용

In [ ]:
# tokenized 열이 추가된 df_result를 csv로.
df_result.to_csv('./data/[Model6]tokenized_added_df_result.csv', encoding = 'utf-8-sig')

## 모델 테스트 용 : 여기서부터 시작~

In [ ]:
df_result = pd.read_csv('./data/[Model6]tokenized_added_df_result.csv', index_col = 0)
df_result

,Date,Title,주가의 날짜,등락률(y),updown,Pos,Neg,Mid,감성지수,tokenized
0,2021-01-04,속보 새해 첫 거래일 코스피 0 04 오른 2874 50 출발,2021-01-05,2.24,1,1,0,0,1,"['속보', '새해', '첫', '거래', '일', '코스피', '0', '04',..."
1,2021-01-04,대웅제약 코로나 치료제 임상3상 시험 시작 주가 상승,2021-01-05,2.24,1,1,0,0,1,"['대웅제약', '코로나', '치료', '제', '임', '상', '3', '상',..."
2,2021-01-04,환율 하락 전환 1086 2 감소0 1원,2021-01-05,2.24,1,0,1,0,-1,"['환율', '하락', '전환', '1086', '2', '감소', '0', '1원']"
3,2021-01-04,비올 일본 최대 병원체인과 실펌엑스 총판계약 체결,2021-01-05,2.24,1,1,0,0,1,"['비올', '일본', '최대', '병원체', '인과', '실펌', '엑스', '총..."
4,2021-01-04,코스닥 하락 전환 968 31 감소0 01,2021-01-05,2.24,1,0,1,0,-1,"['코스닥', '하락', '전환', '968', '31', '감소', '0', '01']"
...,...,...,...,...,...,...,...,...,...,...
431479,2022-06-29,폭락장에 홀로 웃은 OCI 中 신장산 수입규제 반사이익,2022-06-30,-6.67,-1,1,1,0,1,"['폭락', '장', '에', '홀로', '웃은', 'OCI', '中', '신', ..."
431480,2022-06-29,LS 주가 하락장서 역주행 모든 사업부가 호황,2022-06-30,-6.67,-1,1,1,0,1,"['LS', '주가', '하락', '장서', '역주행', '모든', '사업', '부..."
431481,2022-06-29,SK바사 토종 백신 1호 식약처 승인 소식에 3 대 강세,2022-06-30,-6.67,-1,1,0,0,1,"['SK', '바사', '토종', '백신', '1', '호', '식약처', '승인'..."
431482,2022-06-29,장중시황 코스피 1 4 내린 2389선 경기 침체 우려에 약세,2022-06-30,-6.67,-1,0,1,0,-1,"['장', '중', '시', '황', '코스피', '1', '4', '내린', '2..."


In [ ]:
X_train = df_result['tokenized'].values
y_train = df_result['감성지수'].values
X_train

array(["['속보', '새해', '첫', '거래', '일', '코스피', '0', '04', '오른', '2874', '50', '출발']",
       "['대웅제약', '코로나', '치료', '제', '임', '상', '3', '상', '시험', '시작', '주가', '상승']",
       "['환율', '하락', '전환', '1086', '2', '감소', '0', '1원']", ...,
       "['SK', '바사', '토종', '백신', '1', '호', '식약처', '승인', '소식', '에', '3', '대', '강세']",
       "['장', '중', '시', '황', '코스피', '1', '4', '내린', '2389', '선', '경기', '침체', '우려', '에', '약세']",
       "['새빗켐', '증권', '신고', '서', '제출', '코스닥', '상장', '본격', '화']"],
      dtype=object)

In [ ]:
tokenizer = Tokenizer(len(df_result['tokenized']), oov_token = 'OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
max_len = max(len(i) for i in X_train)
print('리뷰 최대 길이 :', max_len)

리뷰 최대 길이 : 42


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 패딩
X_train = pad_sequences(X_train, maxlen = max_len)

In [ ]:
# y label shape 맞춰주기
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, 3)

# LSTM 모델링

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(len(X_train), 100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(3, activation = 'softmax'))

# val_loss를 관찰값으로 설정, 그 값이 감소하는 걸 멈출 때 학습 종료
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose=1, patience=10)

# mc = ModelCheckpoint('real_model6_lstm.h5', monitor = 'val_acc', mode = 'max', verbose = 1, save_best_only=True)
mc = ModelCheckpoint('real_model6_lstm.h5', monitor = 'acc', mode = 'max', verbose = 1, save_best_only=True)


# optimizer 및 loss function 테스트 : 전부 inference 결과가 좋지 않음
# model.compile(optimizer='rmsprop', loss='mse', metrics=['acc'])
# model.compile(optimizer='rmsprop', loss='mae', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='categorical_hinge', metrics = ['acc'])
# model.compile(optimizer='sgd', loss='categorical_hinge', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='hinge', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='KLDivergence', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='LogCosh', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='CosineSimilarity', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='SquaredHinge', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='MeanSquaredLogarithmicError', metrics = ['acc'])
# model.compile(optimizer='rmsprop', loss='huber', metrics = ['acc'])
# model.compile(optimizer='adam', loss='LogCosh', metrics = ['acc'])
# model.compile(optimizer='nadam', loss='LogCosh', metrics = ['acc'])
model.compile(optimizer='adamax', loss='LogCosh', metrics = ['acc'])

history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

model.summary()

Epoch 1/15
1326/1330 [============================>.] - ETA: 0s - loss: 0.0420 - acc: 0.8172
Epoch 1: acc improved from -inf to 0.81736, saving model to real_model6_lstm.h5
1330/1330 [==============================] - 27s 18ms/step - loss: 0.0419 - acc: 0.8174 - val_loss: 0.0379 - val_acc: 0.8493
Epoch 2/15
1329/1330 [============================>.] - ETA: 0s - loss: 0.0225 - acc: 0.9104
Epoch 2: acc improved from 0.81736 to 0.91038, saving model to real_model6_lstm.h5
1330/1330 [==============================] - 22s 17ms/step - loss: 0.0225 - acc: 0.9104 - val_loss: 0.0313 - val_acc: 0.8712
Epoch 3/15
1327/1330 [============================>.] - ETA: 0s - loss: 0.0186 - acc: 0.9247
Epoch 3: acc improved from 0.91038 to 0.92474, saving model to real_model6_lstm.h5
1330/1330 [==============================] - 23s 17ms/step - loss: 0.0186 - acc: 0.9247 - val_loss: 0.0293 - val_acc: 0.8785
Epoch 4/15
1330/1330 [==============================] - ETA: 0s - loss: 0.0169 - acc: 0.9316
Epoch 4

In [ ]:
# 저장된 모델 불러오기, 평가
# loaded_model = load_model('real_model6_lstm.h5')
# loaded_model.evaluate(X_train, y_train)

In [ ]:
y_result = df_result['updown'].values
y_result = to_categorical(y_result, 3)

# 저장된 모델 불러오기, 평가
loaded_model = load_model('real_model6_lstm.h5')
loaded_model.evaluate(X_train, y_result)

13296/13296 [==============================] - 330s 25ms/step - loss: 0.1871 - acc: 0.3250


[0.18706531822681427, 0.32497137784957886]

## 시각화

In [ ]:
# 시각화
hist_dict = history.history
loss = hist_dict['loss']
val_loss = hist_dict['val_loss']
acc = hist_dict['acc']
val_acc = hist_dict['val_acc']

plt.plot(loss, 'b--', label = 'training loss')
plt.plot(val_loss, 'r--', label = 'validation loss')
plt.legend()
plt.grid()

plt.figure()
plt.plot(acc, 'b--', label = 'training accuracy')
plt.plot(val_acc, 'r--', label = 'validation accuracy')
plt.legend()
plt.grid()

plt.show()

NameError: ignored

In [ ]:
import numpy as np

# 임의 불용어
stopwords1 = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords2 = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
stopwords = list(set(stopwords1 + stopwords2))

def sentiment_predict(new_sentence):
    new_token = [word for word in okt.morphs(new_sentence) if not word in stopwords]
    new_sequences = tokenizer.texts_to_sequences([new_token])
    new_pad = pad_sequences(new_sequences, maxlen = max_len)
    # score = float(loaded_model.predict(new_pad))
    score = loaded_model.predict(new_pad)
    # print(score)

    # print(np.argmax(score))

    if np.argmax(score) == 0 :
        print("{} -> 상승 ({:.2f}%)\n".format(new_sentence, score.max() * 100))
    elif np.argmax(score) == 1 :
        print("{} -> 유지 ({:.2f}%)\n".format(new_sentence, score.max() * 100))
    elif np.argmax(score) == 2 :
        print("{} -> 하락 ({:.2f}%)\n".format(new_sentence, score.max() * 100))

In [ ]:
sentiment_predict("[유럽개장] 장초반 상승세…英 1.03%↑")
sentiment_predict("[시황종합] 코스피, '침체우려'에 장중 연저점 경신…2300선 턱걸이 마감")
sentiment_predict("[이번주 증시] 경기침체 우려 지속…반등시 포트폴리오 재정비")
sentiment_predict("'2차전지 너마저'…인플레·경기침체에 장사 없나 [한경우의 케이스스터디]")
sentiment_predict("기관 매도... 연일 증시 폭락")
sentiment_predict("코로나 확진자 3만명에 '비상'…여행-항공주 저가")
sentiment_predict("[특징주]지노믹트리, 핵산 증폭방법 특허 취득·미국 시장 진출 등 소식에 '강세'")
sentiment_predict("러시아 임플란트 수출 반토막에 주가급락한 덴티움…“피크아웃 판단은 시기상조”")

[유럽개장] 장초반 상승세…英 1.03%↑ -> 하락 (55.22%)

[시황종합] 코스피, '침체우려'에 장중 연저점 경신…2300선 턱걸이 마감 -> 하락 (99.02%)

[이번주 증시] 경기침체 우려 지속…반등시 포트폴리오 재정비 -> 하락 (80.61%)

'2차전지 너마저'…인플레·경기침체에 장사 없나 [한경우의 케이스스터디] -> 하락 (84.17%)

기관 매도... 연일 증시 폭락 -> 상승 (84.67%)

코로나 확진자 3만명에 '비상'…여행-항공주 저가 -> 하락 (82.86%)

[특징주]지노믹트리, 핵산 증폭방법 특허 취득·미국 시장 진출 등 소식에 '강세' -> 하락 (98.38%)

러시아 임플란트 수출 반토막에 주가급락한 덴티움…“피크아웃 판단은 시기상조” -> 하락 (75.17%)

